In [53]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor 
import lightgbm as lgb
from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll import scope
from sklearn.model_selection import KFold
from tqdm import tqdm
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.tree import DecisionTreeRegressor

In [110]:
TRAIN = pd.read_excel('C:/Users/user/Downloads/TRAIN.xlsx')

In [111]:
TEST = pd.read_excel('C:/Users/user/Downloads/TEST.xlsx')

In [112]:
new_cols2 = {}
for col in TEST.columns:
    
    if "_​" in col:
        col2 = col.split("_​")
        new_name = ""
        for elem in col2:
            new_name += elem
        
        new_cols2[col] = new_name
    else:
        new_cols2[col] = col
TEST = TEST.rename(new_cols2, axis=1)
TEST

,Item,ANI,Azimuth,Azimuth2,betadep,betaporo,betaporo2,CoreyOW,Coreywater,costeta,...,LNgammaswl2,Major,Major2,Minior,Minior2,Propf,Shiftfacies,Shiftfacies2,Vertical,Vertical2
0,Sector_620,0.083010,21.806383,42.412316,2649.141463,0.188352,0.180913,4.270092,1.230360,0.744077,...,3.863674,4126.053797,6165.275834,629.176607,1080.307651,73.843603,-0.927109,-0.277347,1.375899,13.362094
1,Sector_621,0.087001,12.897296,45.276168,2652.796424,0.180716,0.180803,4.413998,1.459620,0.722829,...,3.883778,3254.387024,5837.662689,330.115937,840.105315,76.672378,-0.917676,-0.272118,4.525007,12.895107
2,Sector_622,0.098740,27.864817,46.912734,2653.043201,0.185998,0.178656,4.300851,1.329981,0.703990,...,3.838583,3958.892384,6199.275855,486.012279,1070.344997,79.162503,-0.907581,-0.131800,1.000000,9.998641
3,Sector_623,0.091231,17.512264,47.793056,2651.120299,0.184644,0.180157,4.568236,1.373982,0.687445,...,3.899891,3396.841340,4860.952721,396.808064,1192.798813,77.266503,-0.643423,-0.096416,6.847620,14.044876
4,Sector_624,0.081904,20.732861,53.867023,2650.296661,0.185642,0.179422,4.428152,1.241696,0.690731,...,3.820225,4190.157138,5659.280189,638.243903,876.080879,78.399370,-0.865536,-0.283815,4.148739,12.567368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Sector_409,0.080025,49.194967,28.878282,2646.519036,0.169797,0.188642,4.396477,2.584249,0.337874,...,3.926670,7687.984863,4626.124607,1339.284514,682.126367,73.467449,-0.408195,-0.240576,22.616185,6.349541
130,Sector_410,0.094979,53.988332,18.974492,2646.147575,0.185567,0.166538,4.334677,2.607186,0.985052,...,3.762498,660.665456,3521.078372,660.665456,1817.236813,92.993708,0.234589,-0.308458,21.130692,11.997013
131,Sector_411,0.094899,48.511348,11.406476,2660.500139,0.182347,0.196942,4.779694,1.311244,0.116778,...,3.886995,7672.278512,4118.644520,1023.010254,955.654276,89.595063,-0.114417,0.157729,12.222095,3.240693
132,Sector_412,0.012559,18.248843,58.450224,2659.933630,0.176094,0.167417,4.125880,1.643645,0.828014,...,3.873257,5880.935850,4577.726829,767.153345,320.177048,90.843173,-0.324235,0.703013,1.219482,20.500844


In [113]:
X_test_global = TEST.copy()
X_test_global = X_test_global.drop(["Item"], axis=1)
X_test_global

,ANI,Azimuth,Azimuth2,betadep,betaporo,betaporo2,CoreyOW,Coreywater,costeta,Fault20,...,LNgammaswl2,Major,Major2,Minior,Minior2,Propf,Shiftfacies,Shiftfacies2,Vertical,Vertical2
0,0.083010,21.806383,42.412316,2649.141463,0.188352,0.180913,4.270092,1.230360,0.744077,0.326171,...,3.863674,4126.053797,6165.275834,629.176607,1080.307651,73.843603,-0.927109,-0.277347,1.375899,13.362094
1,0.087001,12.897296,45.276168,2652.796424,0.180716,0.180803,4.413998,1.459620,0.722829,0.332402,...,3.883778,3254.387024,5837.662689,330.115937,840.105315,76.672378,-0.917676,-0.272118,4.525007,12.895107
2,0.098740,27.864817,46.912734,2653.043201,0.185998,0.178656,4.300851,1.329981,0.703990,0.475669,...,3.838583,3958.892384,6199.275855,486.012279,1070.344997,79.162503,-0.907581,-0.131800,1.000000,9.998641
3,0.091231,17.512264,47.793056,2651.120299,0.184644,0.180157,4.568236,1.373982,0.687445,0.300003,...,3.899891,3396.841340,4860.952721,396.808064,1192.798813,77.266503,-0.643423,-0.096416,6.847620,14.044876
4,0.081904,20.732861,53.867023,2650.296661,0.185642,0.179422,4.428152,1.241696,0.690731,0.423324,...,3.820225,4190.157138,5659.280189,638.243903,876.080879,78.399370,-0.865536,-0.283815,4.148739,12.567368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0.080025,49.194967,28.878282,2646.519036,0.169797,0.188642,4.396477,2.584249,0.337874,0.970553,...,3.926670,7687.984863,4626.124607,1339.284514,682.126367,73.467449,-0.408195,-0.240576,22.616185,6.349541
130,0.094979,53.988332,18.974492,2646.147575,0.185567,0.166538,4.334677,2.607186,0.985052,0.406837,...,3.762498,660.665456,3521.078372,660.665456,1817.236813,92.993708,0.234589,-0.308458,21.130692,11.997013
131,0.094899,48.511348,11.406476,2660.500139,0.182347,0.196942,4.779694,1.311244,0.116778,0.670861,...,3.886995,7672.278512,4118.644520,1023.010254,955.654276,89.595063,-0.114417,0.157729,12.222095,3.240693
132,0.012559,18.248843,58.450224,2659.933630,0.176094,0.167417,4.125880,1.643645,0.828014,0.155373,...,3.873257,5880.935850,4577.726829,767.153345,320.177048,90.843173,-0.324235,0.703013,1.219482,20.500844


In [114]:
new_cols = {}
for col in TRAIN.columns:
    
    if "_​" in col:
        col2 = col.split("_​")
        new_name = ""
        for elem in col2:
            new_name += elem
        
        new_cols[col] = new_name
    else:
        new_cols[col] = col
TRAIN = TRAIN.rename(new_cols, axis=1)
TRAIN

,ANI,Azimuth,Azimuth2,betadep,betaporo,betaporo2,CoreyOW,Coreywater,costeta,Fault20,...,Major,Major2,Minior,Minior2,Propf,Shiftfacies,Shiftfacies2,Vertical,Vertical2,OFPresME10WqH
0,0.023441,52.805802,28.700476,2654.440099,0.183116,0.178074,4.141626,1.976288,0.540317,0.317291,...,5348.309501,2871.325277,990.720243,956.001137,79.336325,0.040837,0.510032,10.031049,14.973673,1743.990729
1,0.069484,33.514137,37.152413,2653.163253,0.183532,0.178910,4.171950,1.697465,0.684603,0.254348,...,4478.663810,3842.056994,665.268861,487.340308,81.064958,-0.575368,0.083334,6.980766,7.238478,1743.880154
2,0.068258,31.224602,29.793483,2653.329520,0.181324,0.181464,4.218976,1.709448,0.671852,0.211177,...,3907.865529,4247.203917,749.749403,600.146336,81.576374,-0.486777,-0.331408,5.705286,6.053614,1642.946108
3,0.065196,35.258320,24.831999,2652.987111,0.184025,0.180014,4.116219,1.601591,0.653267,0.356445,...,3802.755805,4263.122624,688.027599,373.393673,81.558809,-0.354140,-0.121739,8.617207,6.811344,1667.023727
4,0.081851,23.878636,41.585816,2651.434130,0.179994,0.183447,4.090155,1.175829,0.669803,0.000000,...,3946.724684,3385.236541,300.000000,389.070214,81.893010,-0.529016,-0.466060,4.186275,3.822117,1770.329139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.091906,50.931155,34.743838,2662.692191,0.199833,0.176978,3.536307,2.447264,0.089418,0.274347,...,1186.177404,7636.776330,694.544369,698.522986,93.649744,0.186130,-0.156536,12.168036,10.594565,2512.799188
342,0.090692,52.564094,12.994578,2651.607873,0.182777,0.181301,3.681673,1.135026,0.856653,0.194198,...,5581.373028,5942.523728,1130.044659,746.821863,75.550821,0.042074,0.510611,6.616778,6.069723,2181.110571
343,0.038893,27.127944,27.449355,2654.446783,0.179324,0.195223,3.999793,0.437786,0.310680,0.716387,...,2555.155339,4767.191839,633.126916,915.789595,93.732302,0.176035,-0.281000,9.262159,4.185573,1677.391692
344,0.048311,44.860144,31.735425,2646.587221,0.189049,0.191229,4.632672,1.537721,0.894894,0.639470,...,2437.228767,3911.679434,1293.990956,1039.282530,73.769028,0.476189,-0.429017,21.234592,20.018071,2759.100177


In [115]:
range_of_variance = {'ANI': [0.01, 0.1], 'Azimuth': [10, 60], 'Azimuth2': [10, 60], 'betadep': [2645.2, 2663.7], 
'betaporo': [0.164, 0.2],'betaporo2': [0.164, 0.2],'CoreyOW': [3.4, 4.99], 'Coreywater': [0.41, 3.4], 'costeta': [0.071, 0.997],'Fault20': [0.0, 0.997],
 'FWL': [-2671, -2645.0], 'gammadep': [2597.5, 2615.8], 'gammaporo': [0.064, 0.1], 'gammaporo2': [0.064, 0.1],'KrwSorw': [0.102, 0.9], 
'LNbetaperm': [3.35, 5.0], 'LNbetaperm2': [3.35, 5.5], 'LNbetasw': [-3.18, -1.48],'LNbetasw2': [-3.18, -2.0], 
'LNbetaswl': [2.9, 3.3], 'LNbetaswl2': [2.8, 3.3],'LNgammaperm':[-1.38, 0.28], 'LNgammaperm2':[-1.38, 0.1], 'LNgammasw': [-0.39, 0.3],'LNgammasw2': [-0.46, 0.6], 'LNgammaswcr': [1.0, 1.2], 'LNgammaswl': [3.6, 4.0], 'LNgammaswl2': [3.6 , 4.0], 
'Major': [517, 7984], 'Major2': [509, 7989], 'Minior': [300, 1999], 'Minior2': [300, 1993],'Propf': [70, 95], 'Shiftfacies': [-1, 0.1], 'Shiftfacies2': [-1, 0.1], 'Vertical': [1, 25], 'Vertical2': [1, 25]}


def test(range_of_variance):    
    erroneous_columns = {}
    for key_range_of_variance in range_of_variance:
    
        sum_var = (TRAIN[key_range_of_variance] < range_of_variance[key_range_of_variance][0]).sum() + \
              (TRAIN[key_range_of_variance] > range_of_variance[key_range_of_variance][1]).sum()
        
        if sum_var != 0:
            erroneous_columns[key_range_of_variance] = sum_var
           
    print(erroneous_columns)
        
test(range_of_variance)

{'CoreyOW': 2, 'Coreywater': 42, 'Fault20': 1, 'gammadep': 2, 'KrwSorw': 1, 'LNbetaperm2': 1, 'LNbetasw': 2, 'LNbetasw2': 2, 'LNgammaperm': 1, 'LNgammaperm2': 90, 'Shiftfacies': 117, 'Shiftfacies2': 112}


- заменить на средние значения
- проверка на наны
- разделить на тест/трэйн/ вал?
- ласса и выбросить все ненужные параметры
- обучить регрессию лин
- разные регуляризации
- грид серч 
- решающие деревья?

Столбец вариации параметров в этом текстовом документе может содержать ошибку. Не стоит отталкиваться от него при решении задачи, а оценивать распределение из самих данных.

In [116]:
X = TRAIN.copy()
X = X.drop(["OFPresME10WqH"], axis=1)
X

,ANI,Azimuth,Azimuth2,betadep,betaporo,betaporo2,CoreyOW,Coreywater,costeta,Fault20,...,LNgammaswl2,Major,Major2,Minior,Minior2,Propf,Shiftfacies,Shiftfacies2,Vertical,Vertical2
0,0.023441,52.805802,28.700476,2654.440099,0.183116,0.178074,4.141626,1.976288,0.540317,0.317291,...,3.711769,5348.309501,2871.325277,990.720243,956.001137,79.336325,0.040837,0.510032,10.031049,14.973673
1,0.069484,33.514137,37.152413,2653.163253,0.183532,0.178910,4.171950,1.697465,0.684603,0.254348,...,3.833077,4478.663810,3842.056994,665.268861,487.340308,81.064958,-0.575368,0.083334,6.980766,7.238478
2,0.068258,31.224602,29.793483,2653.329520,0.181324,0.181464,4.218976,1.709448,0.671852,0.211177,...,3.859421,3907.865529,4247.203917,749.749403,600.146336,81.576374,-0.486777,-0.331408,5.705286,6.053614
3,0.065196,35.258320,24.831999,2652.987111,0.184025,0.180014,4.116219,1.601591,0.653267,0.356445,...,3.814638,3802.755805,4263.122624,688.027599,373.393673,81.558809,-0.354140,-0.121739,8.617207,6.811344
4,0.081851,23.878636,41.585816,2651.434130,0.179994,0.183447,4.090155,1.175829,0.669803,0.000000,...,3.927696,3946.724684,3385.236541,300.000000,389.070214,81.893010,-0.529016,-0.466060,4.186275,3.822117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,0.091906,50.931155,34.743838,2662.692191,0.199833,0.176978,3.536307,2.447264,0.089418,0.274347,...,3.667304,1186.177404,7636.776330,694.544369,698.522986,93.649744,0.186130,-0.156536,12.168036,10.594565
342,0.090692,52.564094,12.994578,2651.607873,0.182777,0.181301,3.681673,1.135026,0.856653,0.194198,...,3.909973,5581.373028,5942.523728,1130.044659,746.821863,75.550821,0.042074,0.510611,6.616778,6.069723
343,0.038893,27.127944,27.449355,2654.446783,0.179324,0.195223,3.999793,0.437786,0.310680,0.716387,...,3.910752,2555.155339,4767.191839,633.126916,915.789595,93.732302,0.176035,-0.281000,9.262159,4.185573
344,0.048311,44.860144,31.735425,2646.587221,0.189049,0.191229,4.632672,1.537721,0.894894,0.639470,...,3.791832,2437.228767,3911.679434,1293.990956,1039.282530,73.769028,0.476189,-0.429017,21.234592,20.018071


In [117]:
Y = TRAIN["OFPresME10WqH"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

## Линейная регрессия

In [13]:
lin_regr = LinearRegression().fit(X_train, y_train)

In [15]:
y_pred = lin_regr.predict(X_test)

In [21]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse)
print(r2)

118627.35965254906
0.6851145454723659


## Линейная регрессия с регуляризацией
Ridge regression
RidgeRegression is just a LinearRegression, with l2 regularization - penalized for  𝛼⋅∑𝑖𝑤2𝑖
 
Let's train such a model with alpha=0.5

In [79]:
ridge = Ridge(alpha=0.5)
ridge.fit(X_train, y_train)

Ridge(alpha=0.5)

In [80]:
err_test_r = r2_score(y_test, ridge.predict(X_test))
print("Test R2:", err_test_r)

Test R2: 0.6326095341132516


In [81]:
parameter_dict = {
    'alpha': np.exp(np.linspace(-7,5,10))
}
gscv = GridSearchCV(estimator=Ridge(),param_grid=parameter_dict,scoring='neg_mean_squared_error',cv=5)
gscv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': array([9.11881966e-04, 3.45937734e-03, 1.31237287e-02, 4.97870684e-02,
       1.88875603e-01, 7.16531311e-01, 2.71828183e+00, 1.03122585e+01,
       3.91212840e+01, 1.48413159e+02])},
             scoring='neg_mean_squared_error')

In [82]:
gscv.best_params_

{'alpha': 0.0009118819655545162}

In [83]:
best_alpha = gscv.best_params_
best_alpha

{'alpha': 0.0009118819655545162}

In [84]:
best_alpha = list(best_alpha.values())[0]  

In [85]:
ridge = Ridge(alpha = best_alpha)
ridge.fit(X_train, y_train)

Ridge(alpha=0.0009118819655545162)

In [95]:
err_test_r = r2_score(y_test, ridge.predict(X_test))

print("Test R2:", err_test_r)

Test R2: 0.6864338471970393


## Лассо 

In [40]:
parameter_dict = {
    'alpha': np.exp(np.linspace(-7,5,10))
}
gscv = GridSearchCV(estimator=Lasso(),param_grid=parameter_dict,scoring='neg_mean_squared_error',cv=5)
gscv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([9.11881966e-04, 3.45937734e-03, 1.31237287e-02, 4.97870684e-02,
       1.88875603e-01, 7.16531311e-01, 2.71828183e+00, 1.03122585e+01,
       3.91212840e+01, 1.48413159e+02])},
             scoring='neg_mean_squared_error')

In [41]:
gscv.best_params_

{'alpha': 0.18887560283756177}

In [42]:
best_alpha = gscv.best_params_
best_alpha

{'alpha': 0.18887560283756177}

In [43]:
best_alpha = list(best_alpha.values())[0]  

In [44]:
lass = Lasso(alpha = best_alpha)
lass.fit(X_train,y_train)

err_test_l = r2_score(y_test, lass.predict(X_test))

print("Test R2:", err_test_l)

Test R2: 0.6887444631147663


In [49]:
lass.coef_

array([ 3.65008646e+02,  3.61274247e+00, -1.33307065e+00,  4.51855722e+00,
       -1.77950292e+04, -1.61382111e+03,  2.49853086e+01,  2.13938608e+02,
        2.74502928e+02, -1.61974899e+02, -5.21404536e+00,  2.31300576e+00,
       -8.91308082e+02, -0.00000000e+00, -6.86584390e+02, -1.33240628e+02,
       -7.97995530e+01,  1.33985313e+02,  9.70203286e+00,  5.62249152e+02,
       -4.27885172e+01, -1.58930736e+02,  1.16475401e+02,  4.11808268e+01,
        9.65500533e+01,  1.28525636e+02, -8.22937308e+01, -2.94497757e+02,
        1.34954466e-02,  2.01169871e-02, -7.44909674e-04, -2.50307900e-02,
        4.80373492e+00,  9.04056339e+01, -1.32488881e+01,  3.97737413e+00,
        3.03954565e+00])

In [52]:
count = 0
for i in lass.coef_:
    if i != 0:
        count += 1
print("Количество информативных признаков: ", count)
print("Количество НЕ информативных признаков: ", count)

Количество информативных признаков:  36


## Градиентный бустинг

In [119]:
CatBoostRegr = CatBoostRegressor() 
CatBoostRegr.fit(X_train, y_train) 
y_pred = CatBoostRegr.predict(X_test) 

Learning rate set to 0.033407
0:	learn: 617.7334222	total: 41ms	remaining: 41s
1:	learn: 606.9047654	total: 52.8ms	remaining: 26.4s
2:	learn: 595.3193646	total: 63.6ms	remaining: 21.2s
3:	learn: 585.3942963	total: 73.7ms	remaining: 18.4s
4:	learn: 576.3919299	total: 82.8ms	remaining: 16.5s
5:	learn: 566.8034958	total: 91.6ms	remaining: 15.2s
6:	learn: 557.5701614	total: 99.8ms	remaining: 14.2s
7:	learn: 549.2604054	total: 108ms	remaining: 13.4s
8:	learn: 540.8099450	total: 117ms	remaining: 12.9s
9:	learn: 531.8590294	total: 126ms	remaining: 12.5s
10:	learn: 523.7371729	total: 134ms	remaining: 12.1s
11:	learn: 516.8025619	total: 143ms	remaining: 11.8s
12:	learn: 508.7460004	total: 152ms	remaining: 11.5s
13:	learn: 501.6071800	total: 160ms	remaining: 11.2s
14:	learn: 493.6393068	total: 168ms	remaining: 11.1s
15:	learn: 485.7859521	total: 178ms	remaining: 10.9s
16:	learn: 478.8736273	total: 189ms	remaining: 10.9s
17:	learn: 471.8760071	total: 199ms	remaining: 10.8s
18:	learn: 465.4823307	

155:	learn: 163.7116426	total: 1.68s	remaining: 9.09s
156:	learn: 162.8852779	total: 1.69s	remaining: 9.08s
157:	learn: 162.2215322	total: 1.7s	remaining: 9.08s
158:	learn: 161.1478982	total: 1.71s	remaining: 9.06s
159:	learn: 160.2186754	total: 1.72s	remaining: 9.04s
160:	learn: 159.5773049	total: 1.73s	remaining: 9.04s
161:	learn: 158.7966943	total: 1.74s	remaining: 9.02s
162:	learn: 158.3655651	total: 1.75s	remaining: 8.99s
163:	learn: 157.8937347	total: 1.76s	remaining: 8.97s
164:	learn: 157.2822413	total: 1.77s	remaining: 8.96s
165:	learn: 156.6815582	total: 1.78s	remaining: 8.94s
166:	learn: 155.4383518	total: 1.79s	remaining: 8.93s
167:	learn: 154.5711514	total: 1.8s	remaining: 8.93s
168:	learn: 153.7818458	total: 1.81s	remaining: 8.92s
169:	learn: 152.7835678	total: 1.82s	remaining: 8.91s
170:	learn: 151.8855346	total: 1.84s	remaining: 8.9s
171:	learn: 150.8877683	total: 1.85s	remaining: 8.9s
172:	learn: 150.0306146	total: 1.86s	remaining: 8.9s
173:	learn: 149.2458586	total: 1.

320:	learn: 72.3124862	total: 3.53s	remaining: 7.47s
321:	learn: 72.1946096	total: 3.54s	remaining: 7.46s
322:	learn: 72.0471567	total: 3.55s	remaining: 7.45s
323:	learn: 71.5800114	total: 3.56s	remaining: 7.44s
324:	learn: 71.1696323	total: 3.58s	remaining: 7.42s
325:	learn: 71.0504799	total: 3.58s	remaining: 7.41s
326:	learn: 70.5987400	total: 3.59s	remaining: 7.4s
327:	learn: 70.2277886	total: 3.6s	remaining: 7.38s
328:	learn: 69.8148595	total: 3.61s	remaining: 7.37s
329:	learn: 69.4454496	total: 3.62s	remaining: 7.35s
330:	learn: 69.0778086	total: 3.63s	remaining: 7.34s
331:	learn: 68.7168157	total: 3.64s	remaining: 7.32s
332:	learn: 68.6450436	total: 3.65s	remaining: 7.31s
333:	learn: 68.3454693	total: 3.66s	remaining: 7.29s
334:	learn: 68.0071937	total: 3.67s	remaining: 7.28s
335:	learn: 67.8298472	total: 3.67s	remaining: 7.26s
336:	learn: 67.5448845	total: 3.68s	remaining: 7.25s
337:	learn: 67.2167456	total: 3.69s	remaining: 7.23s
338:	learn: 66.8512243	total: 3.7s	remaining: 7.

484:	learn: 35.3268046	total: 5.29s	remaining: 5.61s
485:	learn: 35.1320957	total: 5.3s	remaining: 5.6s
486:	learn: 34.9892218	total: 5.31s	remaining: 5.59s
487:	learn: 34.8949193	total: 5.32s	remaining: 5.58s
488:	learn: 34.8752860	total: 5.33s	remaining: 5.57s
489:	learn: 34.6906392	total: 5.34s	remaining: 5.55s
490:	learn: 34.5440866	total: 5.35s	remaining: 5.54s
491:	learn: 34.4165667	total: 5.36s	remaining: 5.53s
492:	learn: 34.2875762	total: 5.37s	remaining: 5.52s
493:	learn: 34.1540336	total: 5.37s	remaining: 5.5s
494:	learn: 34.0466315	total: 5.38s	remaining: 5.49s
495:	learn: 33.9665213	total: 5.39s	remaining: 5.48s
496:	learn: 33.8257127	total: 5.4s	remaining: 5.46s
497:	learn: 33.6711357	total: 5.41s	remaining: 5.45s
498:	learn: 33.4445096	total: 5.42s	remaining: 5.44s
499:	learn: 33.4238448	total: 5.43s	remaining: 5.43s
500:	learn: 33.2262350	total: 5.43s	remaining: 5.41s
501:	learn: 33.0601706	total: 5.44s	remaining: 5.4s
502:	learn: 32.8701463	total: 5.45s	remaining: 5.39

649:	learn: 18.3782663	total: 6.89s	remaining: 3.71s
650:	learn: 18.3211746	total: 6.9s	remaining: 3.7s
651:	learn: 18.2599211	total: 6.91s	remaining: 3.69s
652:	learn: 18.2014958	total: 6.92s	remaining: 3.68s
653:	learn: 18.1963424	total: 6.93s	remaining: 3.67s
654:	learn: 18.1073839	total: 6.94s	remaining: 3.65s
655:	learn: 18.0421126	total: 6.95s	remaining: 3.65s
656:	learn: 17.9698094	total: 6.96s	remaining: 3.63s
657:	learn: 17.9614682	total: 6.97s	remaining: 3.62s
658:	learn: 17.9534138	total: 6.98s	remaining: 3.61s
659:	learn: 17.9133888	total: 6.99s	remaining: 3.6s
660:	learn: 17.8382661	total: 7s	remaining: 3.59s
661:	learn: 17.7676131	total: 7.01s	remaining: 3.58s
662:	learn: 17.6828007	total: 7.02s	remaining: 3.57s
663:	learn: 17.5830568	total: 7.06s	remaining: 3.57s
664:	learn: 17.5683458	total: 7.07s	remaining: 3.56s
665:	learn: 17.4877158	total: 7.08s	remaining: 3.55s
666:	learn: 17.4265098	total: 7.09s	remaining: 3.54s
667:	learn: 17.3675637	total: 7.1s	remaining: 3.53s


814:	learn: 9.8166818	total: 8.5s	remaining: 1.93s
815:	learn: 9.7565577	total: 8.51s	remaining: 1.92s
816:	learn: 9.7227637	total: 8.52s	remaining: 1.91s
817:	learn: 9.7196604	total: 8.53s	remaining: 1.9s
818:	learn: 9.6645471	total: 8.54s	remaining: 1.89s
819:	learn: 9.6263766	total: 8.55s	remaining: 1.88s
820:	learn: 9.6044483	total: 8.55s	remaining: 1.86s
821:	learn: 9.5435872	total: 8.56s	remaining: 1.85s
822:	learn: 9.4906960	total: 8.57s	remaining: 1.84s
823:	learn: 9.4388365	total: 8.58s	remaining: 1.83s
824:	learn: 9.4016011	total: 8.59s	remaining: 1.82s
825:	learn: 9.3505797	total: 8.6s	remaining: 1.81s
826:	learn: 9.3479083	total: 8.61s	remaining: 1.8s
827:	learn: 9.2814508	total: 8.62s	remaining: 1.79s
828:	learn: 9.2780989	total: 8.63s	remaining: 1.78s
829:	learn: 9.2501995	total: 8.63s	remaining: 1.77s
830:	learn: 9.2172425	total: 8.64s	remaining: 1.76s
831:	learn: 9.1945892	total: 8.65s	remaining: 1.75s
832:	learn: 9.1556914	total: 8.66s	remaining: 1.74s
833:	learn: 9.10

978:	learn: 5.2534588	total: 10.1s	remaining: 217ms
979:	learn: 5.2253646	total: 10.1s	remaining: 207ms
980:	learn: 5.2103950	total: 10.1s	remaining: 196ms
981:	learn: 5.1852446	total: 10.2s	remaining: 186ms
982:	learn: 5.1683234	total: 10.2s	remaining: 176ms
983:	learn: 5.1552601	total: 10.2s	remaining: 165ms
984:	learn: 5.1272252	total: 10.2s	remaining: 155ms
985:	learn: 5.1092724	total: 10.2s	remaining: 145ms
986:	learn: 5.0911392	total: 10.2s	remaining: 134ms
987:	learn: 5.0883502	total: 10.2s	remaining: 124ms
988:	learn: 5.0853866	total: 10.2s	remaining: 114ms
989:	learn: 5.0641763	total: 10.2s	remaining: 103ms
990:	learn: 5.0476012	total: 10.2s	remaining: 93ms
991:	learn: 5.0167961	total: 10.2s	remaining: 82.6ms
992:	learn: 4.9929129	total: 10.3s	remaining: 72.3ms
993:	learn: 4.9795441	total: 10.3s	remaining: 62ms
994:	learn: 4.9779455	total: 10.3s	remaining: 51.6ms
995:	learn: 4.9569240	total: 10.3s	remaining: 41.3ms
996:	learn: 4.9285627	total: 10.3s	remaining: 31ms
997:	learn:

In [120]:
err_test_r = r2_score(y_test, CatBoostRegr.predict(X_test))

print("Test R2:", err_test_r)

Test R2: 0.7198165338609783


In [ ]:
0.7198165338609783 ()

## ПРЕДСКАЗАНИЕ НА TEST :


In [124]:
y_test_pred = CatBoostRegr.predict(X_test_global) 
y_test_pred

array([1591.11836739, 1587.53283387, 1534.35666298, 1607.85219349,
       1590.69848296, 1575.33804916, 1598.47754818, 1541.02784225,
       1605.20493828, 1499.67288098, 1521.2986185 , 1542.23575125,
       1572.94703009, 1622.03139476, 1496.18262873, 1610.76302541,
       1536.09849834, 1574.01870591, 1705.79432018, 1694.93830323,
       1716.43498447, 1678.98006312, 1528.82078017, 1561.16627869,
       1611.65624127, 1715.79752605, 1583.64314845, 1659.47415995,
       1521.95367946, 1600.04018995, 1497.46101026, 1660.27837391,
       1562.60708802, 1698.3205404 , 1612.49127423, 1597.24988114,
       1750.32541232, 1677.01884709, 1658.07800468, 1569.37882251,
       1531.30303292, 1562.34852103, 1782.27432183, 1667.29642667,
       1704.63635501, 1564.59902478, 1647.19634834, 1686.09943768,
       1686.32094433, 1725.22090405, 1650.65959574, 1656.8837217 ,
       1704.15858023, 1728.09532067, 1568.72785529, 1596.58542793,
       1683.3279407 , 1680.81651225, 1789.79237157, 1690.65729

In [125]:
len(y_test_pred)

134

## Catboost + KFold

In [103]:
space = {'max_depth': hp.choice('max_depth', [7, 8, 9]),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),'n_estimators': hp.choice('n_estimators', np.arange(2, 2000, 500, dtype=int))
}

def evaluate_catboost(params):    
    kf = KFold(n_splits=5, shuffle=True)
    for train_ids, val_ids in kf.split(X_train):
        curr_X_train = X_train.iloc[train_ids]
        curr_y_train = y_train.iloc[train_ids] 
        curr_X_val = X_train.iloc[val_ids]        
        curr_y_val = y_train.iloc[val_ids]
        clf = CatBoostRegressor(**params, verbose=0)
        clf.fit(curr_X_train, curr_y_train)
        y_pred = clf.predict(curr_X_val)
        rmse = np.sqrt(mean_squared_error(curr_y_val, y_pred)) 
        
    return {'loss': rmse, 'status': STATUS_OK}

trials = Trials()
best_params = fmin(fn = evaluate_catboost,
                   space = space, algo = tpe.suggest, 
                   max_evals = 100,
                   trials=trials)

print('Best parameters:', best_params)

100%|█████████████████████████████████████████████| 100/100 [2:40:58<00:00, 96.59s/trial, best loss: 267.3577294082493]
Best parameters: {'learning_rate': 0.27906025134052936, 'max_depth': 0, 'n_estimators': 3}


In [118]:
CatBoostRegr = CatBoostRegressor(learning_rate=0.27, max_depth=3, n_estimators=1700, ) 
CatBoostRegr.fit(X_train, y_train) 

0:	learn: 550.5583978	total: 5.04ms	remaining: 8.57s
1:	learn: 487.9964719	total: 9.97ms	remaining: 8.46s
2:	learn: 452.9298574	total: 13ms	remaining: 7.35s
3:	learn: 410.0768705	total: 15.6ms	remaining: 6.59s
4:	learn: 381.5285988	total: 18.1ms	remaining: 6.13s
5:	learn: 364.0043267	total: 20.9ms	remaining: 5.89s
6:	learn: 347.1527783	total: 25.5ms	remaining: 6.16s
7:	learn: 332.5208801	total: 29.5ms	remaining: 6.24s
8:	learn: 318.1804091	total: 33.4ms	remaining: 6.27s
9:	learn: 307.8040207	total: 36.5ms	remaining: 6.17s
10:	learn: 303.2744409	total: 39.8ms	remaining: 6.12s
11:	learn: 297.1499842	total: 43.8ms	remaining: 6.17s
12:	learn: 291.0905543	total: 47.6ms	remaining: 6.17s
13:	learn: 283.9958548	total: 51.2ms	remaining: 6.17s
14:	learn: 279.8246742	total: 53.4ms	remaining: 6s
15:	learn: 272.6266270	total: 56.1ms	remaining: 5.9s
16:	learn: 263.7182366	total: 58.4ms	remaining: 5.78s
17:	learn: 259.4808122	total: 60.5ms	remaining: 5.65s
18:	learn: 254.8900579	total: 62.8ms	remaini

171:	learn: 29.8225156	total: 355ms	remaining: 3.15s
172:	learn: 29.0905764	total: 363ms	remaining: 3.2s
173:	learn: 28.7431296	total: 368ms	remaining: 3.23s
174:	learn: 28.2012378	total: 374ms	remaining: 3.25s
175:	learn: 27.8414739	total: 377ms	remaining: 3.27s
176:	learn: 27.4295651	total: 382ms	remaining: 3.28s
177:	learn: 26.9639130	total: 385ms	remaining: 3.29s
178:	learn: 26.5912153	total: 388ms	remaining: 3.3s
179:	learn: 26.4254454	total: 392ms	remaining: 3.31s
180:	learn: 26.1436931	total: 395ms	remaining: 3.32s
181:	learn: 25.5591792	total: 399ms	remaining: 3.33s
182:	learn: 25.2744017	total: 403ms	remaining: 3.34s
183:	learn: 24.9857165	total: 406ms	remaining: 3.35s
184:	learn: 24.5669853	total: 408ms	remaining: 3.35s
185:	learn: 24.0789993	total: 411ms	remaining: 3.34s
186:	learn: 23.8900515	total: 413ms	remaining: 3.35s
187:	learn: 23.5104396	total: 416ms	remaining: 3.34s
188:	learn: 23.2153220	total: 419ms	remaining: 3.35s
189:	learn: 23.0568231	total: 421ms	remaining: 3

343:	learn: 4.4971975	total: 697ms	remaining: 2.75s
344:	learn: 4.4530541	total: 701ms	remaining: 2.75s
345:	learn: 4.3298482	total: 705ms	remaining: 2.76s
346:	learn: 4.2623276	total: 708ms	remaining: 2.76s
347:	learn: 4.2109674	total: 712ms	remaining: 2.77s
348:	learn: 4.1529632	total: 716ms	remaining: 2.77s
349:	learn: 4.1440299	total: 720ms	remaining: 2.78s
350:	learn: 4.1320885	total: 724ms	remaining: 2.78s
351:	learn: 4.1209677	total: 728ms	remaining: 2.79s
352:	learn: 4.0852570	total: 730ms	remaining: 2.79s
353:	learn: 4.0499218	total: 733ms	remaining: 2.79s
354:	learn: 4.0140102	total: 735ms	remaining: 2.79s
355:	learn: 3.9771855	total: 738ms	remaining: 2.79s
356:	learn: 3.9734216	total: 740ms	remaining: 2.78s
357:	learn: 3.8584016	total: 742ms	remaining: 2.78s
358:	learn: 3.8322850	total: 744ms	remaining: 2.78s
359:	learn: 3.8230256	total: 746ms	remaining: 2.78s
360:	learn: 3.7693848	total: 748ms	remaining: 2.77s
361:	learn: 3.7380313	total: 750ms	remaining: 2.77s
362:	learn: 

596:	learn: 0.4000497	total: 1.23s	remaining: 2.28s
597:	learn: 0.3962958	total: 1.23s	remaining: 2.27s
598:	learn: 0.3944341	total: 1.24s	remaining: 2.27s
599:	learn: 0.3914491	total: 1.24s	remaining: 2.27s
600:	learn: 0.3867981	total: 1.24s	remaining: 2.27s
601:	learn: 0.3816348	total: 1.24s	remaining: 2.26s
602:	learn: 0.3768347	total: 1.24s	remaining: 2.26s
603:	learn: 0.3756774	total: 1.25s	remaining: 2.26s
604:	learn: 0.3729286	total: 1.25s	remaining: 2.26s
605:	learn: 0.3725621	total: 1.25s	remaining: 2.25s
606:	learn: 0.3707350	total: 1.25s	remaining: 2.25s
607:	learn: 0.3695780	total: 1.25s	remaining: 2.25s
608:	learn: 0.3659514	total: 1.25s	remaining: 2.25s
609:	learn: 0.3616350	total: 1.25s	remaining: 2.24s
610:	learn: 0.3582046	total: 1.26s	remaining: 2.24s
611:	learn: 0.3577348	total: 1.26s	remaining: 2.24s
612:	learn: 0.3545888	total: 1.26s	remaining: 2.24s
613:	learn: 0.3528868	total: 1.26s	remaining: 2.23s
614:	learn: 0.3463787	total: 1.26s	remaining: 2.23s
615:	learn: 

779:	learn: 0.0764003	total: 1.61s	remaining: 1.9s
780:	learn: 0.0761784	total: 1.62s	remaining: 1.9s
781:	learn: 0.0759950	total: 1.62s	remaining: 1.9s
782:	learn: 0.0748276	total: 1.62s	remaining: 1.9s
783:	learn: 0.0738652	total: 1.62s	remaining: 1.9s
784:	learn: 0.0731272	total: 1.63s	remaining: 1.9s
785:	learn: 0.0729679	total: 1.63s	remaining: 1.9s
786:	learn: 0.0720139	total: 1.63s	remaining: 1.9s
787:	learn: 0.0707658	total: 1.64s	remaining: 1.89s
788:	learn: 0.0695984	total: 1.64s	remaining: 1.89s
789:	learn: 0.0691695	total: 1.64s	remaining: 1.89s
790:	learn: 0.0678096	total: 1.64s	remaining: 1.89s
791:	learn: 0.0675061	total: 1.64s	remaining: 1.89s
792:	learn: 0.0670266	total: 1.65s	remaining: 1.88s
793:	learn: 0.0662782	total: 1.65s	remaining: 1.88s
794:	learn: 0.0658101	total: 1.65s	remaining: 1.88s
795:	learn: 0.0651262	total: 1.65s	remaining: 1.88s
796:	learn: 0.0645825	total: 1.65s	remaining: 1.87s
797:	learn: 0.0644745	total: 1.65s	remaining: 1.87s
798:	learn: 0.064056

1033:	learn: 0.0084061	total: 2.22s	remaining: 1.43s
1034:	learn: 0.0083168	total: 2.23s	remaining: 1.43s
1035:	learn: 0.0082454	total: 2.23s	remaining: 1.43s
1036:	learn: 0.0082332	total: 2.23s	remaining: 1.43s
1037:	learn: 0.0082117	total: 2.23s	remaining: 1.42s
1038:	learn: 0.0081666	total: 2.23s	remaining: 1.42s
1039:	learn: 0.0081243	total: 2.23s	remaining: 1.42s
1040:	learn: 0.0081132	total: 2.24s	remaining: 1.42s
1041:	learn: 0.0081010	total: 2.24s	remaining: 1.41s
1042:	learn: 0.0080796	total: 2.24s	remaining: 1.41s
1043:	learn: 0.0079239	total: 2.24s	remaining: 1.41s
1044:	learn: 0.0078577	total: 2.24s	remaining: 1.41s
1045:	learn: 0.0078531	total: 2.25s	remaining: 1.4s
1046:	learn: 0.0077718	total: 2.25s	remaining: 1.4s
1047:	learn: 0.0077664	total: 2.25s	remaining: 1.4s
1048:	learn: 0.0076976	total: 2.25s	remaining: 1.4s
1049:	learn: 0.0076207	total: 2.26s	remaining: 1.4s
1050:	learn: 0.0075786	total: 2.26s	remaining: 1.39s
1051:	learn: 0.0074624	total: 2.26s	remaining: 1.39

1199:	learn: 0.0020584	total: 2.55s	remaining: 1.06s
1200:	learn: 0.0020560	total: 2.55s	remaining: 1.06s
1201:	learn: 0.0020345	total: 2.55s	remaining: 1.06s
1202:	learn: 0.0019899	total: 2.55s	remaining: 1.05s
1203:	learn: 0.0019871	total: 2.56s	remaining: 1.05s
1204:	learn: 0.0019786	total: 2.56s	remaining: 1.05s
1205:	learn: 0.0019768	total: 2.56s	remaining: 1.05s
1206:	learn: 0.0019722	total: 2.56s	remaining: 1.04s
1207:	learn: 0.0019573	total: 2.56s	remaining: 1.04s
1208:	learn: 0.0019312	total: 2.56s	remaining: 1.04s
1209:	learn: 0.0018922	total: 2.57s	remaining: 1.04s
1210:	learn: 0.0018777	total: 2.57s	remaining: 1.04s
1211:	learn: 0.0018573	total: 2.57s	remaining: 1.03s
1212:	learn: 0.0018474	total: 2.57s	remaining: 1.03s
1213:	learn: 0.0018264	total: 2.57s	remaining: 1.03s
1214:	learn: 0.0018021	total: 2.58s	remaining: 1.03s
1215:	learn: 0.0017877	total: 2.58s	remaining: 1.03s
1216:	learn: 0.0017861	total: 2.58s	remaining: 1.02s
1217:	learn: 0.0017826	total: 2.58s	remaining:

1402:	learn: 0.0003752	total: 2.9s	remaining: 614ms
1403:	learn: 0.0003676	total: 2.9s	remaining: 612ms
1404:	learn: 0.0003656	total: 2.9s	remaining: 610ms
1405:	learn: 0.0003604	total: 2.91s	remaining: 608ms
1406:	learn: 0.0003588	total: 2.91s	remaining: 606ms
1407:	learn: 0.0003573	total: 2.91s	remaining: 604ms
1408:	learn: 0.0003521	total: 2.91s	remaining: 602ms
1409:	learn: 0.0003516	total: 2.92s	remaining: 600ms
1410:	learn: 0.0003458	total: 2.92s	remaining: 598ms
1411:	learn: 0.0003450	total: 2.92s	remaining: 596ms
1412:	learn: 0.0003446	total: 2.92s	remaining: 594ms
1413:	learn: 0.0003444	total: 2.92s	remaining: 592ms
1414:	learn: 0.0003388	total: 2.93s	remaining: 590ms
1415:	learn: 0.0003379	total: 2.93s	remaining: 588ms
1416:	learn: 0.0003330	total: 2.93s	remaining: 586ms
1417:	learn: 0.0003275	total: 2.93s	remaining: 584ms
1418:	learn: 0.0003270	total: 2.94s	remaining: 582ms
1419:	learn: 0.0003216	total: 2.94s	remaining: 580ms
1420:	learn: 0.0003197	total: 2.94s	remaining: 57

1653:	learn: 0.0000395	total: 3.44s	remaining: 95.7ms
1654:	learn: 0.0000394	total: 3.44s	remaining: 93.7ms
1655:	learn: 0.0000393	total: 3.45s	remaining: 91.6ms
1656:	learn: 0.0000392	total: 3.45s	remaining: 89.5ms
1657:	learn: 0.0000389	total: 3.45s	remaining: 87.4ms
1658:	learn: 0.0000388	total: 3.45s	remaining: 85.3ms
1659:	learn: 0.0000384	total: 3.45s	remaining: 83.2ms
1660:	learn: 0.0000379	total: 3.46s	remaining: 81.1ms
1661:	learn: 0.0000375	total: 3.46s	remaining: 79ms
1662:	learn: 0.0000375	total: 3.46s	remaining: 77ms
1663:	learn: 0.0000372	total: 3.46s	remaining: 74.9ms
1664:	learn: 0.0000365	total: 3.46s	remaining: 72.8ms
1665:	learn: 0.0000365	total: 3.46s	remaining: 70.7ms
1666:	learn: 0.0000364	total: 3.46s	remaining: 68.6ms
1667:	learn: 0.0000361	total: 3.47s	remaining: 66.5ms
1668:	learn: 0.0000360	total: 3.47s	remaining: 64.4ms
1669:	learn: 0.0000359	total: 3.47s	remaining: 62.4ms
1670:	learn: 0.0000356	total: 3.47s	remaining: 60.3ms
1671:	learn: 0.0000354	total: 3.

In [119]:
y_pred = CatBoostRegr.predict(X_test) 
err_test_r = r2_score(y_test, CatBoostRegr.predict(X_test))
print("Test R2:", err_test_r)

Test R2: 0.7554082058420264


In [120]:
y_test_pred = CatBoostRegr.predict(X_test_global) 
y_test_pred

array([1503.13301925, 1573.73709316, 1513.28758728, 1517.10494866,
       1734.38124751, 1571.27625507, 1635.39958395, 1590.16745418,
       1746.35752412, 1459.7332655 , 1626.9099178 , 1474.54647924,
       1553.9745549 , 1844.50232925, 1491.25301795, 1523.75421643,
       1618.80587508, 1726.0017751 , 1582.56332319, 1921.45988781,
       1891.02902747, 1765.84469485, 1564.23505403, 1566.95744226,
       1694.96738388, 1772.06267609, 1455.29901854, 1786.53670643,
       1529.00815902, 1620.24631156, 1467.04716747, 1709.75316894,
       1400.12877825, 1899.25169067, 1626.55398115, 1582.17141451,
       1833.98161973, 1731.86734591, 1754.59898618, 1639.25977135,
       1391.13695218, 1632.39199994, 1676.96944891, 1635.24824821,
       1756.34118275, 1521.69336166, 1757.61153062, 1726.68031129,
       1753.37403526, 1625.83456907, 1646.10492374, 1791.48563064,
       1857.04571394, 1713.05272387, 1448.58022357, 1543.88287335,
       1888.65558622, 1602.12117699, 1839.84405084, 1759.25715

In [105]:
res = pd.DataFrame({'predict': np.round(y_test_pred, 5)})
res

,predict
0,1503.13302
1,1573.73709
2,1513.28759
3,1517.10495
4,1734.38125
...,...
129,2745.31471
130,2390.38315
131,2486.58713
132,2545.43450


## lightgbm

In [82]:
params = {
    'task': 'train', 
    'n_estimators': 1700,
    'boosting': 'gbdt',
    'objective': 'regression',
    'learnnig_rage': 0.27,
    'max_depth': 3,
    'metric': {'l2','l1'},
    'verbose': -1
}

In [83]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [84]:
model = lgb.train(params,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=30)

[LightGBM] [Warning] Unknown parameter: learnnig_rage
[1]	valid_0's l1: 505.43	valid_0's l2: 329716
Training until validation scores don't improve for 30 rounds
[2]	valid_0's l1: 472.657	valid_0's l2: 291211
[3]	valid_0's l1: 444.203	valid_0's l2: 260401
[4]	valid_0's l1: 419.841	valid_0's l2: 236239
[5]	valid_0's l1: 397.824	valid_0's l2: 216437
[6]	valid_0's l1: 378.201	valid_0's l2: 201866
[7]	valid_0's l1: 363.314	valid_0's l2: 191939
[8]	valid_0's l1: 348.866	valid_0's l2: 182138
[9]	valid_0's l1: 334.285	valid_0's l2: 173485
[10]	valid_0's l1: 320.448	valid_0's l2: 165697
[11]	valid_0's l1: 310.35	valid_0's l2: 159611
[12]	valid_0's l1: 301.962	valid_0's l2: 156525
[13]	valid_0's l1: 293.88	valid_0's l2: 150930
[14]	valid_0's l1: 287.203	valid_0's l2: 148838
[15]	valid_0's l1: 283.6	valid_0's l2: 148808
[16]	valid_0's l1: 278.664	valid_0's l2: 147178
[17]	valid_0's l1: 274.767	valid_0's l2: 146218
[18]	valid_0's l1: 267.778	valid_0's l2: 143211
[19]	valid_0's l1: 264.06	valid_0's

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\users\user\appdata\local\programs\python\python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


In [85]:
y_pred = model.predict(X_test)

# accuracy check
r2 = r2_score(y_test, y_pred)

print("R2: %.2f", r2) 

R2: %.2f 0.6678156685375274


## DecisionTreeRegressor

In [112]:
DT_regr = DecisionTreeRegressor()
DT_regr.fit(X_train, y_train)

DecisionTreeRegressor()

In [113]:
y_pred = DT_regr.predict(X_test) 
err_test_r = r2_score(y_test, DT_regr.predict(X_test))
print("Test R2:", err_test_r)

Test R2: 0.3122166233137429
